In [7]:
from qiskit import QuantumCircuit, transpile, execute
from qiskit.providers.fake_provider import FakeCairo, FakeKolkata, FakeMontreal
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.result import LocalReadoutMitigator
import pickle
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import qiskit.providers.aer.noise as noise
import numpy as np
import json
from qiskit import pulse


class exp_runner:
    def __init__(self, model, amp_thresh=0.4):
        self.model = (
            model  # name of the error model, can be "cairo", "kolkata", "montreal"
        )
        self.amp_thresh = amp_thresh  # threshold for pauli string pruning
        self.get_backend()
        self.get_circ_and_obs()

    def get_backend(self):
        # get the backend and mitigator for the backend
        if self.model == "cairo":
            backend = FakeCairo()
            with open("NoiseModel/fakecairo.pkl", "rb") as f:
                noise_model = pickle.load(f)
            layout = [1,2,4,5,6,8,9,10,11,18,22,26]  # best layout that minimize noise
        if self.model == "kolkata":
            backend = FakeKolkata()
            with open("NoiseModel/fakekolkata.pkl", "rb") as f:
                noise_model = pickle.load(f)
            layout = [2, 3, 8, 9, 10, 12, 14, 16, 18, 19, 22, 26]
        if self.model == "montreal":
            backend = FakeMontreal()
            with open("NoiseModel/fakemontreal.pkl", "rb") as f:
                noise_model = pickle.load(f)
            layout = [0, 3, 5, 10, 11, 13, 14, 18, 21, 23, 24, 25]
        noise_model = noise.NoiseModel.from_dict(noise_model)
        simulator = AerSimulator(noise_model=noise_model)
        mitigator = LocalReadoutMitigator(backend=backend, qubits=layout)
        self.backend = backend
        self.noise_model = noise_model
        self.simulator = simulator
        self.mitigator = mitigator
        self.layout = layout
        self.total_shot = 0  # a counter for the total number of shots used

    def get_circ_and_obs(self):
        # get the ansatz circuit and the shots for each pauli string
        with open("Hamiltonian/OHhamiltonian_groups.json", "r") as f:
            groups = json.load(
                f
            )  # dict of dict, the first key is the superobservable, the second key is the sub-observable, the value is the coefficient
        obs = groups.keys()
        # a subobservable is a pauli string that can be generated by the original pauli string by replacing some of the pauli operators with I
        subobs = groups
        shots_dict = {}
        # how many shots to use for each superobservable
        total_amp = 0

        for ob in obs:
            amp = 0
            for subob in subobs[ob]:
                amp += subobs[ob][subob] ** 2
            amp = amp**0.5
            if amp > self.amp_thresh:
                total_amp += amp
        for ob in obs:
            amp = 0
            for subob in subobs[ob]:
                amp += subobs[ob][subob] ** 2
            amp = amp**0.5
            if amp > self.amp_thresh:
                shots_dict[ob] = np.floor(1800000 * amp / total_amp)
            else:
                shots_dict[ob] = 0
        self.obs = obs
        self.shots_dict = shots_dict
        self.subobs = subobs
        with open("ansatz.qasm", "r") as f:
            ansatz = f.read()
            circ = QuantumCircuit.from_qasm_str(ansatz)
        self.circ = circ

    def diagonal_and_run(self, circ, obs, shots, seed):
        # diagonalize the circuit and sample from it
        new_circ = circ.copy()
        for i in range(12):
            if obs[i] == "X":
                new_circ.h(i)
            elif obs[i] == "Y":
                new_circ.sdg(i)
                new_circ.h(i)
            elif obs[i] == "Z":
                pass
            else:
                pass
        new_circ.measure_all()
        result = (
            execute(
                new_circ,
                self.simulator,
                initial_layout=self.layout,
                shots=shots,
                seed_simulator=seed,
            )
            .result()
            .get_counts()
        )
        self.total_shot += shots
        return result

    def construct_diagonal_matrix(self, subob):
        # construct the diagonal matrix for a sub-observable
        diagonal = np.ones(1)
        for i in range(12):
            if subob[i] != "I":
                diagonal = np.kron(np.array([1, -1]), diagonal)
            else:
                diagonal = np.kron(np.array([1, 1]), diagonal)
        return diagonal

    def calc_exp_val(self, circ, shots_adjustment=1, seed=0):
        # calculate the expectation value of the ansatz circuit, ignoring identity pauli strings
        exp_val = 0
        for ob in self.obs:
            shots = self.shots_dict[ob]
            if shots == 0: # ignore pauli strings that have amplitude smaller than the threshold
                continue
            data = self.diagonal_and_run(circ, ob, shots * shots_adjustment, seed)
            diagonal = np.zeros(2**12)
            for subob in self.subobs[ob].keys():
                diagonal += (
                    self.construct_diagonal_matrix(subob) * self.subobs[ob][subob]
                )
            val = self.mitigator.expectation_value(data, diagonal=diagonal)[0]
            exp_val += val
        return exp_val

    def run(self, seed):
        # calculate the expectation value using ZNE, this is our method
        self.total_shot = 0
        circ = self.circ.copy()
        circ = transpile(circ, self.simulator)

        org_exp_val = self.calc_exp_val(circ, shots_adjustment=1.5 / 2, seed=seed) # use 75% of the shots for the original circuit

        new_circ = self.circ.copy()
        new_circ.barrier()
        new_circ.extend(self.circ.inverse())
        new_circ.barrier()
        new_circ.extend(self.circ)
        new_exp_val = self.calc_exp_val(new_circ, shots_adjustment=0.5 / 2, seed=seed) # use 25% of the shots for the new circuit
        return (
            1.5 * org_exp_val - 0.5 * new_exp_val - 51.1139071673799 + 4.36537496654537
        )

    def no_ZNE(self, seed):
        # calculate the expectation value without ZNE, this is used for comparison
        circ = self.circ.copy()
        circ = transpile(circ, self.simulator)
        org_exp_val = self.calc_exp_val(circ, shots_adjustment=1, seed=seed)
        return org_exp_val - 51.1139071673799 + 4.36537496654537

    def get_pulse_duration(self):
        with pulse.build(self.backend) as my_program1:
            with pulse.transpiler_settings(optimization_level=0):
                pulse.call(self.circ)
        print(f"pulse duration:{my_program1.duration}")

    def get_total_shot(self):
        print(f"total shots:{self.total_shot}")


def calc_score(result):
    print(f"Score:{(1-np.abs(-74.38714627 - np.mean(result))/74.38714627)*100}")


def calc_variance(result):
    print(f"Variance:{np.std(result)}")

In [8]:
seed_list=[20, 21, 30, 33, 36, 42, 43, 55, 67, 170]

In [9]:
model = "cairo"
runner = exp_runner(model)
result_list = []
for seed in seed_list:
    result_list.append(runner.run(seed))
runner.get_pulse_duration()
runner.get_total_shot()
calc_score(result_list)
calc_variance(result_list)

pulse duration:96
total shots:1800000.0
Score:99.89092044595634
Variance:0.002954454427173438


In [10]:
model = "kolkata"
runner = exp_runner(model)
result_list = []
for seed in seed_list:
    result_list.append(runner.run(seed))
runner.get_pulse_duration()
runner.get_total_shot()
calc_score(result_list)
calc_variance(result_list)

pulse duration:160
total shots:1800000.0
Score:99.88718060201029
Variance:0.0021725095261954236


In [11]:
model = "montreal"
runner = exp_runner(model)
result_list = []
for seed in seed_list:
    result_list.append(runner.run(seed))
runner.get_pulse_duration()
runner.get_total_shot()
calc_score(result_list)
calc_variance(result_list)

pulse duration:160
total shots:1800000.0
Score:99.88647132711903
Variance:0.0027439212028582656
